In [1]:
from tokenizers import Tokenizer
from models import Model_Adam
from layers import Embedding,PositionalEncoding,TransformerBlock,Dense,LayerNorm,softmax
import numpy as np
from datasets import load_dataset
from itertools import islice

d:\App\Python1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pip install tokenizers datasets

In [13]:
tokenizer = Tokenizer.from_file(r"D:\Workspace\Python\Project\NeuralNetwork\dataset\tokenizer\tokenizer_vi.json")

dataset = load_dataset("justinphan3110/vi_pubmed", split="vi", streaming=True)
dataset_iter = iter(islice(dataset, 64*3000))
pad_token = '<|pad|>'
maxlen = 64
pad_id = tokenizer.token_to_id(pad_token)
def split_token_batch(texts):
        x, y = [], []
    
        # batch encode
        encodings = tokenizer.encode_batch(texts)
    
        for encoding in encodings:
            token = encoding.ids
    
            if len(token) <= maxlen + 1:
                token = token + (maxlen+1-len(token))*[pad_id]
                x.append(token[:-1])
                y.append(token[1:])
                
    
            else:
                x.append(token[:maxlen])
                y.append(token[1:maxlen + 1])
        return x, y
def data_generator(texts):
    X, Y = split_token_batch([str(t['text']) for t in texts])
    return np.array(X, dtype=np.int32), np.array(Y, dtype=np.int32)


# batch_texts = list(islice(dataset_iter, 64*3))
# X,y = data_generator(batch_texts)

In [4]:
def create_causal_mask(batch_size, seq_len):
    """Create causal mask for autoregressive generation"""
    mask = np.ones((batch_size, seq_len, seq_len))
    return np.tril(mask)

In [27]:


class MyGPTModel(Model_Adam):
    """GPT-style autoregressive transformer model"""
    def __init__(self, vocab_size=10000, max_len=256, embedding_dim=768,
                 n_heads=12, n_layers=12):
        pad_token = '<|pad|>'
        pad_id = tokenizer.token_to_id(pad_token)
        super().__init__()
        # super().__init__(vocab_size=vocab_size, row_num=1_000_000, maxlen=max_len, pad_id = pad_id)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.max_len = max_len
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.d_ff = 4 * embedding_dim  
        
        self.token_embedding = Embedding(vocab_size, embedding_dim)
        self.pos_encoding = PositionalEncoding(max_len, embedding_dim)
        self.transformer_blocks = [
            TransformerBlock(embedding_dim, n_heads, self.d_ff)
            for _ in range(n_layers)
        ]
        
        self.ln_f = LayerNorm(embedding_dim)

        self.lm_head = Dense(vocab_size, activation='softmax')
        # self.lm_head = Dense_Attention(vocab_size, activation='softmax')
    def call(self, input_ids,training=False):
        batch_size, seq_len = input_ids.shape       

        x = self.token_embedding(input_ids)
        x = self.pos_encoding(x,training=False)
        
        causal_mask = create_causal_mask(batch_size, seq_len)
        
        for block in self.transformer_blocks:
            x = block.forward(x, mask=causal_mask)
        
        x = self.ln_f(x)
        
        # logits = self.lm_head(x)
        # print(training)
        if training:
            logits = self.lm_head(x)           # (batch, seq_len, vocab_size)
        else:
            logits = self.lm_head(x[:, -1:, :]) # (batch, vocab_size) - last token
        
        return logits
    
    def _all_layer(self):
        """Override to collect all layers including transformer blocks"""
        layers = []

        layers.append(self.token_embedding)
        layers.extend(self.transformer_blocks)
        layers.append(self.ln_f)
        layers.append(self.lm_head)
        
        return layers

model = MyGPTModel(
    vocab_size=10000,
    embedding_dim=128,      
    max_len=64,            
    n_heads=4,                 
    n_layers=1,
)

# model.fit(X, y, learning_rate=1e-3, epochs=1, batch_size=16)

In [30]:
# batch_texts = list(islice(dataset_iter, 64*3000))

from tqdm import tqdm
pbar = tqdm(range(0,3000*64))
for t in pbar:
    
    text = batch_texts[t]['text']
    
    tokens = tokenizer.encode(text).ids
    # Không thêm chiều batch sớm
    x_tokens = tokens[:-1]
    y_tokens = tokens[1:]

    # Sau đó mới tạo mảng và reshape thành (1, seq_len)
    X = np.array(x_tokens, dtype=np.int32).reshape(1, -1)
    y = np.array(y_tokens, dtype=np.int32).reshape(1, -1)

    model.fit(X, y, learning_rate=1e-3, epochs=1, batch_size=1 ,verbose=False)
    

  1%|          | 1081/192000 [05:26<16:01:17,  3.31it/s]


KeyboardInterrupt: 

In [32]:
class TextGenerator:
    def __init__(self, max_tokens, tokenizer, model,top_k=1, temperature=1.0):
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.k = top_k
        self.temperature = temperature
        self.model = model
        # self.maxlen = maxlen
        pad_token = '<|pad|>'
        self.pad_id = tokenizer.token_to_id(pad_token)
    def softmax(self, logits):
        logits = np.asarray(logits)  # Chuyển về numpy array
        return softmax(logits / self.temperature)
    
    def sample_from(self, logits):
        """Top-k sampling from logits (1D array)"""
        logits = np.asarray(logits).astype(np.float32)  # Đảm bảo numpy
        top_k_indices = np.argpartition(-logits, self.k)[:self.k]
        top_k_logits = logits[top_k_indices]
        probs = self.softmax(top_k_logits)
        
        # Chuyển về CPU để dùng numpy.random.choice
        probs_cpu = np.asarray(probs)
        sampled_pos = np.random.choice(len(top_k_indices), size=1, p=probs_cpu)[0]
        return int(np.asarray(top_k_indices[sampled_pos]))  # Chuyển về int
    
    def generate(self, prompt):
        start_tokens = self.tokenizer.encode(prompt).ids
        token_ids = start_tokens.copy()
        
        for _ in range(self.max_tokens):
            # if len(token_ids) < self.maxlen:
            #     input_ids = token_ids + [self.pad_id] * (self.maxlen - len(token_ids))
            # else:
            #     input_ids = token_ids[-self.maxlen:]
            input_ids = token_ids
            # Tạo numpy array
            input_array = np.array([input_ids], dtype=np.int32)  # (1, seq_len)
            logits = self.model(input_array)     # Output shape: (1, seq_len, vocab_size)
            # print(logits.shape)
            last_pos = len(token_ids) - 1
            # if last_pos >= self.maxlen:
            #     last_pos = self.maxlen - 1
            
            next_logits = logits[0,0]   # Shape: (vocab_size,)
            next_id = self.sample_from(next_logits)
            
            token_ids.append(next_id)
        
        return self.tokenizer.decode(token_ids, skip_special_tokens=False)
        
generator = TextGenerator(
    max_tokens=640,
    tokenizer=tokenizer,
    model=model,
    top_k=1,
    temperature=0.9,
)

q = 'anh'
text = generator.generate(q)
print(text+"|")


anh cứu cứu cứu của các của các cứu, được nghiên cứu, được nghiên cứu, các cứu, được nghiên cứu. Các cứu, được nghiên cứu. Sự. Sự. Sự. Sự. Các cứu. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự,. Sự. Sự. Sự. Sự. Sự. Sự. Sự,. Sự. Các.2,,,,. Các. Sự,. Sự,. Sự,. Sự. Sự. Sự, và. Sự. Sự. Sự. Sự. Sự. Sự.2,.2,.2,,,,.2,,. Sự,. Sự, được trị, được trị, các cứu, pH, pH, pH, pH, cứu cứu, các cứu,,,. Sự,. Sự,. Sự,,,,,,,. Sự,. Sự,. Sự. Sự. Sự. Sự,. Sự. Sự, các cứu,. Sự. Sự. Sự. Sự. Sự. Sự, được trị,. Sự, các. Sự,. Sự,. Sự. Sự. Sự. Sự. Sự. Sự,,,,,,, và các trị,,,,, và các trị, các trị, và các các các độ,. Sự, các. Sự,, các các các các. Sự,,,,,,,.yme, với với với với các với các với. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự,. Sự,,,,,,,,,,,,,,,,. Sự. Sự,. Sự. Sự trị, sự trị. Sự trị. Sự trị. Sự. Sự. Sự. Sự. Sự. Sự. Sự. Sự và. Sự. Sự. Sự. Sự. các.ate.ate. các trị. các. các..... nhân. nhân. nhân,. các.ic,,,,,,,.. độ. độ thể,.. nhân,,..ate,............................ với,,,,,,,,,,,,,,,,